In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **문제 1**

 입력 이미지가 15x15x3일 때 다음 각각에 대해 출력데이터 형태와 매개변수의 개수를 구하시오. 
 (단, stride=1, padding은 없음, 각 문항은 연계과정이 아닌 별도의 개별적인 처리과정)

**(1)(5,5) 크기의 filter 한 개를 한 번 적용할 때**

필터공식을 적용한다. ans: 11x11x1, 5x5x3+1=76

**(2) (3,3) 크기의 filter 한 개를 두 번 적용(Conv2D를 두 번 적용)할 때**

필터공식을 적용한다. ans: 13x13x2, (3x3x3+1)x2=56

**(3) (3,3) 크기의 filter 8개로 convolution 층(Conv2D)을 적용할 때**

필터공식을 적용한다. ans: 13x13x8, (3x3x3+1)x8=224  

**(4) (3)의 결과를 MaxPooling2D를 옵션 없이 적용할 때 출력데이터의 형태를 구하시오.**

ans 6x6x8, (2x2의 필터로 풀링을 한다. 나머지는 자른다.)

# **문제 2:cifar10 dataset으로 CNN 모델 학습**

# **(1) 데이터 불러오기**

In [2]:
from tensorflow.keras.datasets.cifar10 import load_data
(train_x, train_y), (test_x, test_y) = load_data()

In [3]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape  #이미지사이즈;32x32x3

In [4]:
np.unique(train_y,return_counts=True) 
#lable(train_y)을 unique를 사용해 조사 0~9 10개의 클래스, 분포:각각500개

#10개의 클래스에 다중분류이다

# **(2) 데이터 전처리 : /255**

In [5]:
train_x=train_x/255
test_x=test_x/255

# **(3) CNN모델 라이브러리 불러오기**

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

# **(4) CNN모델 설정: Conv2D, MaxPooling2D, Dense 등**

In [7]:
CNN=Sequential()
CNN.add(Conv2D(32,(3, 3), activation='relu', input_shape=(32,32,3,)))
CNN.add(MaxPooling2D())
CNN.add(Conv2D(16,(3,3),activation='relu'))
CNN.add(MaxPooling2D())
CNN.add(Conv2D(8,(2,2),activation='relu'))
CNN.add(MaxPooling2D())
CNN.add(Flatten())
CNN.add(Dense(32, activation='relu'))
CNN.add(Dense(16, activation='relu'))
CNN.add(Dense(10, activation= 'softmax'))
CNN.summary()

In [8]:
from tensorflow.keras.utils import plot_model
plot_model(CNN, 'CNN.png')

# **(5) complie 설정**

In [9]:
CNN.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics='acc')

# **(6) fit (학습 진행) (validation_split=0.1)**

In [10]:
hist=CNN.fit(train_x, train_y, epochs=80, batch_size=45, validation_split=0.1)

# **(7) loss와 val_loss 그래프 그리기**

In [11]:
loss=hist.history['loss']          #손실
val_loss=hist.history['val_loss']  # validation 에 대한 손실

acc=hist.history['acc']            #정확도
val_acc=hist.history['val_acc']    #validation 에 대한 정확도

ep=np.arange(1, len(loss)+1)   #인덱스

import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
plt.plot(ep,loss,'b', label='loss')
plt.plot(ep,val_loss,'g', label='val_loss')
plt.title('Loss')
plt.legend()

# **(8) acc와 val_acc 그래프 그리기**

In [12]:
plt.figure(figsize=(10,8))
plt.plot(ep,acc,'b', label='acc')
plt.plot(ep,val_acc,'g', label='val_acc')
plt.title('Accuracy')
plt.legend()

# **(9) test data evaluate**

In [13]:
CNN.evaluate(test_x,test_y)